In [1]:
import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [2]:
input_default_json_conf_file='cnn_emotion4_training_yolo_default.json';

In [3]:
#import numpy as np
#import pandas as pd
#import tensorflow as tf
#import datetime
import shutil
import json

In [4]:
import sys
sys.path.append('../library');

In [5]:
#print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [6]:
## Load json conf json file
fd = open(os.path.join('./',input_default_json_conf_file));
DATA = json.load(fd);
fd.close()

# Variable globales

In [7]:
## Seed for the random variables
seed_number=0;

## Dataset 
dataset_dir    = DATA['dataset_dir'];
dataset_name   = DATA['dataset_name'];

## Training hyperparameters
EPOCAS     = DATA["epochs"];
BATCH_SIZE = DATA["batch_size"];

## Model of network
## 'yolov8n-cls','yolov8s-cls','yolov8m-cls'
model_type = DATA["model_type"];

## Output
output_base_dir = DATA["output_base_dir"];


# Parametros de entrada

In [8]:
for n in range(len(sys.argv)):
    if sys.argv[n]=='--dataset-dir':
        dataset_dir=sys.argv[n+1];
    elif sys.argv[n]=='--dataset-name':
        dataset_name=sys.argv[n+1];
    elif sys.argv[n]=='--model':
        model_type=sys.argv[n+1];
    elif sys.argv[n]=='--epochs':
        EPOCAS=int(sys.argv[n+1]);
    elif sys.argv[n]=='--batch-size':
        BATCH_SIZE=int(sys.argv[n+1]);
    elif sys.argv[n]=='--output-dir':
        output_base_dir=sys.argv[n+1];
        
print('    dataset_dir:',dataset_dir)
print('   dataset_name:',dataset_name)
print('     model_type:',model_type)
print('         EPOCAS:',EPOCAS)
print('     BATCH_SIZE:',BATCH_SIZE)
print('output_base_dir:',output_base_dir)

    dataset_dir: /media/fernando/Expansion/DATASET/YOLO-COPY/TESE/BER/BER2024/BER2024-BODY
   dataset_name: ber2024-body
     model_type: yolov8n-cls
         EPOCAS: 2
     BATCH_SIZE: 8
output_base_dir: /media/fernando/Expansion/OUTPUTS/DOCTORADO2/cnn_emotion4_yolo


# Set seed of random variables


In [9]:
#np.random.seed(seed_number)
#tf.keras.utils.set_random_seed(seed_number);

# Creating output directory

In [10]:
output_dir = os.path.join(output_base_dir,dataset_name,'training_validation_holdout_fine_tuning',model_type);

os.makedirs(output_dir,exist_ok = True);

print(output_dir)

/media/fernando/Expansion/OUTPUTS/DOCTORADO2/cnn_emotion4_yolo/ber2024-body/training_validation_holdout/yolov8n-cls


# Create new model

In [11]:
import BodyEmotion4Lib.lib_yolo_model as myp

model, target_size = myp.create_model(model_type=model_type,load_weights=False,file_of_weight='');


myp.save_model_parameters(model, os.path.join(output_dir,'parameters_stats.m'));


YOLOv8n-cls summary: 99 layers, 2719288 parameters, 2719288 gradients, 4.4 GFLOPs
Transferred 158/158 items from pretrained weights
Loading architecture yolov8n-cls

        url: yolov8n-cls.yaml
target_size: (224, 224)



# Train and validation


In [12]:
# https://docs.ultralytics.com/modes/train/#train-settings
results = model.train(data=dataset_dir, epochs=EPOCAS, imgsz=target_size,batch=BATCH_SIZE,save=True,exist_ok=True)

best_model_file=os.path.join('runs','classify','train','weights','best.pt');



New https://pypi.org/project/ultralytics/8.2.15 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.10 🚀 Python-3.8.10 torch-1.11.0+cu102 CUDA:0 (NVIDIA GeForce MX150, 2003MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=classify, mode=train, model=yolov8n-cls.yaml, data=/media/fernando/Expansion/DATASET/YOLO-COPY/TESE/BER/BER2024/BER2024-BODY, epochs=2, time=None, patience=100, batch=8, imgsz=(224, 224), save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, strea

train: Scanning /media/fernando/Expansion/DATASET/YOLO-COPY/TESE/BER/BER2024/BER2024-BODY/train... 21594 images, 0 corrupt: 100%|██████████| 21594/21594 [00:00<?, ?it/s]
/home/fernando/.local/lib/python3.8/site-packages/torchvision/transforms/transforms.py:890: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
/home/fernando/.local/lib/python3.8/site-packages/torchvision/transforms/functional.py:1139: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
/home/fernando/.local/lib/python3.8/site-packages/torchvision/transforms/functional.py:1139: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
/home/fernando/.local/lib/python3.8/site-packages/torchvision/transforms/functional.py:1050: UserWarning: Argument interpolation 

optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000714, momentum=0.9) with parameter groups 26 weight(decay=0.0), 27 weight(decay=0.0005), 27 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 224 train, 224 val
Using 8 dataloader workers
Logging results to runs/classify/train
Starting training for 2 epochs...

      Epoch    GPU_mem       loss  Instances       Size


  0%|          | 0/2700 [00:00<?, ?it/s]/home/fernando/.local/lib/python3.8/site-packages/torchvision/transforms/functional.py:1050: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
        1/2     0.103G      1.539          8        224:   0%|          | 0/2700 [00:00<?, ?it/s]/home/fernando/.local/lib/python3.8/site-packages/torchvision/transforms/functional.py:1050: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
        1/2     0.113G      1.465          8        224:   1%|          | 18/2700 [00:02<08:45,  5.10it/s]/home/fernando/.local/lib/python3.8/site-packages/torchvision/transforms/functional.py:1050: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
        1/2     0.113G      1.405          8        224

KeyboardInterrupt: 

# Evaluate best model

In [ ]:
from tqdm import tqdm
def verify_categorical_accuracy(model,input_dir,labels_dic={0: 'negative', 1: 'neutro', 2: 'pain', 3: 'positive'}):
    L=0;
    Count=0;
    print('\nWorking on',input_dir);
    for ID, LABEL in labels_dic.items():
        dir_path=os.path.join(input_dir,LABEL);
        print('Working on subdirectory',LABEL,'...')
        archivo_list=os.listdir(dir_path);
        L=L+len(archivo_list);
        for n in tqdm(range(len(archivo_list))):
            res=model(os.path.join(dir_path,archivo_list[n]),verbose=False);
            if ID==res[0].probs.top1:
                Count=Count+1;
    return Count*1.0/L, L;  

In [ ]:
# LOAD BEST MODEL to evaluate the performance of the model
model=model.load(best_model_file);

train_acc, L_train = verify_categorical_accuracy(model,os.path.join(dataset_dir,'train'));
print('train_acc:',train_acc);

val_acc  , L_val   = verify_categorical_accuracy(model,os.path.join(dataset_dir,'val'));
print('val_acc:',val_acc);

test_acc , L_test  = verify_categorical_accuracy(model,os.path.join(dataset_dir,'test'));
print('test_acc:',test_acc);

In [ ]:

data_results=dict();
data_results['train_categorical_accuracy']=train_acc;
data_results['val_categorical_accuracy']  =val_acc;
data_results['test_categorical_accuracy'] =test_acc;
data_results['train_length']=L_train;
data_results['val_length']=L_val;
data_results['test_length']=L_test;
data_results['number_of_parameters']=myp.get_model_parameters(model);
print('data',data_results);

In [ ]:
# final all json
with open(os.path.join(output_dir,"training_data_results.json"), 'w') as f:
    json.dump(data_results, f,indent=4);

# final test txt
with open(os.path.join(output_dir,"results_testing.txt"), 'w') as f: 
    for key, value in data_results.items(): 
        f.write('%s=%s;\n' % (key, value));


In [ ]:
tmp_name='model_'+model_type+'.pt';

shutil.copy(best_model_file,os.path.join(output_dir,tmp_name));

print('moving','runs')
shutil.move('runs', os.path.join(output_dir,'runs'));
print('moved')